# Trabajo Final - Parte 3: Análisis de Políticas Públicas
## Visualización de Datos - Latinobarómetro 2024

Este notebook contiene el análisis de dos preguntas de investigación relevantes para políticas públicas, explorando relaciones y correlaciones entre variables del Latinobarómetro 2024.


In [ ]:
# Instalación de dependencias necesarias
!pip install pandas numpy matplotlib seaborn plotly scipy -q


In [ ]:
# Importación de librerías
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# Configuración de estilo para visualizaciones
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (14, 8)
plt.rcParams['font.size'] = 11

# Para mostrar todas las columnas en pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_rows', 100)


## 1. Carga y Preparación de Datos


In [ ]:
# Carga de datos principales
df = pd.read_csv('../data/Latinobarometro_2024.csv', sep=';', encoding='utf-8')

# Carga de códigos de países
country_codes = pd.read_csv('../data/country_codes.csv', encoding='utf-8')

# Merge con códigos de países
df = df.merge(country_codes, left_on='IDENPA', right_on='Country Code', how='left')
df['Country Name'] = df['Country Name'].fillna('Unknown')

print(f"Dimensiones del dataset: {df.shape}")
print(f"Países incluidos: {df['Country Name'].nunique()}")
print(f"\nPaíses y número de encuestados:")
print(df['Country Name'].value_counts().sort_index())


In [ ]:
# Preparación de variables principales

# Variables demográficas
df['sexo_label'] = df['SEXO'].map({1: 'Hombre', 2: 'Mujer'})
df['grupo_edad'] = pd.cut(df['EDAD'], bins=[0, 25, 35, 45, 55, 65, 100], 
                          labels=['16-25', '26-35', '36-45', '46-55', '56-65', '66+'])

# Educación
df['anos_estudio'] = df['REEDUC.3'].replace([-1, -2, -3, -5, 96, 97, 98, 99], np.nan)
df['grupo_educativo'] = pd.cut(df['anos_estudio'], bins=[0, 6, 12, 99], 
                                labels=['Primaria (0-6)', 'Secundaria (7-12)', 'Universidad (13+)'],
                                include_lowest=True)

# Confianza en instituciones (S14M.1 = Gobierno)
df['confianza_gobierno'] = df['S14M.1'].replace([-1, -2, -3, -5, 96, 97, 98, 99], np.nan)

# Satisfacción con la democracia (S16)
df['satisfaccion_democracia'] = df['S16'].replace([-1, -2, -3, -4, -5, 96, 97, 98, 99], np.nan)

# Interés en política (S17)
df['interes_politica'] = df['S17'].replace([-1, -2, -3, -5, 96, 97, 98, 99], np.nan)

# Acceso a internet (S7: 1=Sí, 2=No)
df['acceso_internet'] = df['S7'].replace([-1, -2, -3, -5, 96, 97, 98, 99], np.nan)
df['tiene_internet'] = df['acceso_internet'].map({1: 'Sí', 2: 'No'})

# Confianza en redes sociales (P23STM.2)
df['confianza_redes'] = df['P23STM.2'].replace([-1, -2, -3, -5, 96, 97, 98, 99], np.nan)

# Uso de redes sociales (P28ST)
df['uso_redes'] = df['P28ST'].replace([-1, -2, -3, -5, 96, 97, 98, 99], np.nan)
# Mapear valores según codebook (asumiendo: 1=Diario, 2=Semanal, 3=Nunca)
df['uso_redes_label'] = df['uso_redes'].map({1: 'Diario', 2: 'Semanal', 3: 'Nunca'})

print("Variables preparadas exitosamente")
print(f"\nRegistros válidos por variable:")
print(f"  - Años de estudio: {df['anos_estudio'].notna().sum()}")
print(f"  - Confianza en gobierno: {df['confianza_gobierno'].notna().sum()}")
print(f"  - Satisfacción democracia: {df['satisfaccion_democracia'].notna().sum()}")
print(f"  - Interés en política: {df['interes_politica'].notna().sum()}")
print(f"  - Acceso internet: {df['acceso_internet'].notna().sum()}")
print(f"  - Confianza redes: {df['confianza_redes'].notna().sum()}")
print(f"  - Uso redes: {df['uso_redes'].notna().sum()}")


## 2. Preguntas de Investigación para Políticas Públicas

### 2.1. Pregunta 1: Educación y Participación Política

**Pregunta de investigación:**
¿Cómo se relaciona el nivel educativo con la confianza en instituciones gubernamentales y la satisfacción con la democracia en América Latina?

**Justificación:**
Esta pregunta es relevante para políticas públicas porque:
1. **Políticas educativas**: Comprender la relación entre educación y participación política puede informar sobre la importancia de la educación cívica y la formación ciudadana.
2. **Fortalecimiento democrático**: La confianza en instituciones es fundamental para la estabilidad democrática. Si la educación está correlacionada con mayor confianza, esto justifica inversiones en educación.
3. **Desigualdad política**: Si existe una brecha entre niveles educativos en términos de participación y confianza, esto puede indicar la necesidad de políticas inclusivas.
4. **Efectividad de políticas**: Los gobiernos necesitan entender qué factores influyen en la legitimidad percibida de sus instituciones.

**Variables a analizar:**
- Variable independiente: Nivel educativo (años de estudio, grupos educativos)
- Variables dependientes: Confianza en gobierno, satisfacción con la democracia, interés en política
- Variables de control: País, edad, sexo

### 2.2. Pregunta 2: Brecha Digital y Participación Política

**Pregunta de investigación:**
¿Existe una relación entre el acceso a internet, el uso de redes sociales y la participación política (interés en política, confianza en gobierno) en América Latina?

**Justificación:**
Esta pregunta es relevante para políticas públicas porque:
1. **Políticas de inclusión digital**: La brecha digital puede estar creando nuevas formas de exclusión política. Si el acceso a internet está correlacionado con mayor participación, esto justifica políticas de conectividad.
2. **Democracia digital**: Los gobiernos están migrando servicios y comunicación a plataformas digitales. Entender quién queda excluido es crucial.
3. **Desinformación y confianza**: Las redes sociales pueden influir en la confianza en instituciones. Comprender esta relación ayuda a diseñar políticas de alfabetización mediática.
4. **Participación ciudadana**: Si las redes sociales facilitan la participación política, esto tiene implicaciones para políticas de gobierno abierto y transparencia.

**Variables a analizar:**
- Variables independientes: Acceso a internet, uso de redes sociales, confianza en redes sociales
- Variables dependientes: Interés en política, confianza en gobierno, satisfacción con la democracia
- Variables de control: País, nivel educativo, edad, sexo


## 3. Análisis de la Pregunta 1: Educación y Participación Política

### 3.1. Análisis Descriptivo por Nivel Educativo


### 3.2. Análisis por País: Comparación Regional


In [ ]:
# Análisis de correlación entre educación y confianza por país
pais_educ_analisis = df.groupby(['Country Name', 'grupo_educativo']).agg({
    'confianza_gobierno': 'mean',
    'anos_estudio': 'mean'
}).reset_index()

# Crear matriz de correlación por país
correlaciones_pais = []
for pais in df['Country Name'].unique():
    df_pais = df[df['Country Name'] == pais]
    corr = df_pais[['anos_estudio', 'confianza_gobierno']].corr().iloc[0, 1]
    if not np.isnan(corr):
        correlaciones_pais.append({'País': pais, 'Correlación': corr})

corr_df = pd.DataFrame(correlaciones_pais).sort_values('Correlación', ascending=False)
print("Correlación entre años de estudio y confianza en gobierno por país:")
print(corr_df.to_string(index=False))


In [ ]:
# Visualización: Heatmap de confianza por país y nivel educativo
pivot_confianza = df.pivot_table(
    values='confianza_gobierno', 
    index='Country Name', 
    columns='grupo_educativo', 
    aggfunc='mean'
)

plt.figure(figsize=(12, 10))
sns.heatmap(pivot_confianza, annot=True, fmt='.2f', cmap='RdYlGn', 
            cbar_kws={'label': 'Confianza Promedio (1-10)'}, 
            linewidths=0.5, linecolor='gray')
plt.title('Confianza en Gobierno: País vs Nivel Educativo', 
          fontweight='bold', fontsize=14, pad=20)
plt.xlabel('Nivel Educativo', fontweight='bold')
plt.ylabel('País', fontweight='bold')
plt.tight_layout()
plt.show()


### 3.3. Análisis de Correlación y Regresión


In [ ]:
# Análisis de correlación general
variables_educ = ['anos_estudio', 'confianza_gobierno', 'satisfaccion_democracia', 'interes_politica']
corr_matrix = df[variables_educ].corr()

print("Matriz de correlación:")
print(corr_matrix.round(3))

# Test de significancia estadística
print("\n\nCorrelación entre años de estudio y confianza en gobierno:")
df_clean = df[['anos_estudio', 'confianza_gobierno']].dropna()
if len(df_clean) > 0:
    corr_coef, p_value = stats.pearsonr(df_clean['anos_estudio'], df_clean['confianza_gobierno'])
    print(f"  Coeficiente de correlación: {corr_coef:.4f}")
    print(f"  Valor p: {p_value:.4f}")
    print(f"  Significativo (p < 0.05): {'Sí' if p_value < 0.05 else 'No'}")


In [ ]:
# Visualización: Scatter plot con línea de regresión
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# 1. Educación vs Confianza en gobierno
df_clean1 = df[['anos_estudio', 'confianza_gobierno']].dropna()
axes[0].scatter(df_clean1['anos_estudio'], df_clean1['confianza_gobierno'], 
                alpha=0.3, s=10, color='steelblue')
z = np.polyfit(df_clean1['anos_estudio'], df_clean1['confianza_gobierno'], 1)
p = np.poly1d(z)
axes[0].plot(df_clean1['anos_estudio'], p(df_clean1['anos_estudio']), 
             "r--", linewidth=2, label=f'Tendencia (r={stats.pearsonr(df_clean1["anos_estudio"], df_clean1["confianza_gobierno"])[0]:.3f})')
axes[0].set_xlabel('Años de Estudio', fontweight='bold')
axes[0].set_ylabel('Confianza en Gobierno (1-10)', fontweight='bold')
axes[0].set_title('Relación entre Educación y Confianza en Gobierno', fontweight='bold', fontsize=12)
axes[0].legend()
axes[0].grid(alpha=0.3)

# 2. Educación vs Satisfacción con democracia
df_clean2 = df[['anos_estudio', 'satisfaccion_democracia']].dropna()
axes[1].scatter(df_clean2['anos_estudio'], df_clean2['satisfaccion_democracia'], 
                alpha=0.3, s=10, color='coral')
z2 = np.polyfit(df_clean2['anos_estudio'], df_clean2['satisfaccion_democracia'], 1)
p2 = np.poly1d(z2)
axes[1].plot(df_clean2['anos_estudio'], p2(df_clean2['anos_estudio']), 
             "r--", linewidth=2, label=f'Tendencia (r={stats.pearsonr(df_clean2["anos_estudio"], df_clean2["satisfaccion_democracia"])[0]:.3f})')
axes[1].set_xlabel('Años de Estudio', fontweight='bold')
axes[1].set_ylabel('Satisfacción con Democracia (1-10)', fontweight='bold')
axes[1].set_title('Relación entre Educación y Satisfacción con Democracia', fontweight='bold', fontsize=12)
axes[1].legend()
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()


## 4. Análisis de la Pregunta 2: Brecha Digital y Participación Política

### 4.1. Análisis Descriptivo: Acceso a Internet y Uso de Redes


In [ ]:
# Estadísticas por acceso a internet
internet_stats = df.groupby('tiene_internet').agg({
    'interes_politica': ['mean', 'std', 'count'],
    'confianza_gobierno': ['mean', 'std'],
    'satisfaccion_democracia': ['mean', 'std']
}).round(2)

print("Estadísticas por acceso a internet:")
print(internet_stats)

print("\n\nEstadísticas por uso de redes sociales:")
redes_stats = df.groupby('uso_redes_label').agg({
    'interes_politica': ['mean', 'std', 'count'],
    'confianza_gobierno': ['mean', 'std'],
    'confianza_redes': ['mean', 'std']
}).round(2)
print(redes_stats)


In [ ]:
# Visualización: Comparación entre quienes tienen y no tienen internet
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# 1. Interés en política por acceso a internet
internet_interes = df.groupby('tiene_internet')['interes_politica'].mean()
axes[0, 0].bar(internet_interes.index, internet_interes.values, 
               color=['#e74c3c', '#27ae60'], alpha=0.8, edgecolor='black', linewidth=1.5)
axes[0, 0].set_ylabel('Interés Promedio (1-4)', fontweight='bold')
axes[0, 0].set_title('Interés en Política por Acceso a Internet', fontweight='bold', fontsize=12)
axes[0, 0].grid(axis='y', alpha=0.3)
for i, v in enumerate(internet_interes.values):
    axes[0, 0].text(i, v, f'\n{v:.2f}', ha='center', fontweight='bold', fontsize=11)

# 2. Confianza en gobierno por acceso a internet
internet_confianza = df.groupby('tiene_internet')['confianza_gobierno'].mean()
axes[0, 1].bar(internet_confianza.index, internet_confianza.values, 
               color=['#e74c3c', '#27ae60'], alpha=0.8, edgecolor='black', linewidth=1.5)
axes[0, 1].set_ylabel('Confianza Promedio (1-10)', fontweight='bold')
axes[0, 1].set_title('Confianza en Gobierno por Acceso a Internet', fontweight='bold', fontsize=12)
axes[0, 1].grid(axis='y', alpha=0.3)
for i, v in enumerate(internet_confianza.values):
    axes[0, 1].text(i, v, f'\n{v:.2f}', ha='center', fontweight='bold', fontsize=11)

# 3. Interés en política por uso de redes sociales
redes_interes = df.groupby('uso_redes_label')['interes_politica'].mean()
orden_redes = ['Diario', 'Semanal', 'Nunca']
redes_interes_ordenado = redes_interes.reindex([x for x in orden_redes if x in redes_interes.index])
axes[1, 0].bar(redes_interes_ordenado.index, redes_interes_ordenado.values, 
               color=['#3498db', '#9b59b6', '#e67e22'], alpha=0.8, edgecolor='black', linewidth=1.5)
axes[1, 0].set_ylabel('Interés Promedio (1-4)', fontweight='bold')
axes[1, 0].set_title('Interés en Política por Uso de Redes Sociales', fontweight='bold', fontsize=12)
axes[1, 0].grid(axis='y', alpha=0.3)
for i, v in enumerate(redes_interes_ordenado.values):
    axes[1, 0].text(i, v, f'\n{v:.2f}', ha='center', fontweight='bold', fontsize=11)

# 4. Confianza en gobierno por uso de redes sociales
redes_confianza = df.groupby('uso_redes_label')['confianza_gobierno'].mean()
redes_confianza_ordenado = redes_confianza.reindex([x for x in orden_redes if x in redes_confianza.index])
axes[1, 1].bar(redes_confianza_ordenado.index, redes_confianza_ordenado.values, 
               color=['#3498db', '#9b59b6', '#e67e22'], alpha=0.8, edgecolor='black', linewidth=1.5)
axes[1, 1].set_ylabel('Confianza Promedio (1-10)', fontweight='bold')
axes[1, 1].set_title('Confianza en Gobierno por Uso de Redes Sociales', fontweight='bold', fontsize=12)
axes[1, 1].grid(axis='y', alpha=0.3)
for i, v in enumerate(redes_confianza_ordenado.values):
    axes[1, 1].text(i, v, f'\n{v:.2f}', ha='center', fontweight='bold', fontsize=11)

plt.tight_layout()
plt.show()


### 4.2. Análisis por País: Brecha Digital Regional


In [ ]:
# Porcentaje de acceso a internet por país
acceso_por_pais = df.groupby('Country Name')['tiene_internet'].value_counts(normalize=True).unstack() * 100
acceso_por_pais = acceso_por_pais.sort_values('Sí', ascending=False)

print("Porcentaje de acceso a internet por país:")
print(acceso_por_pais.round(1))

# Diferencia en interés político entre quienes tienen y no tienen internet por país
diferencias = []
for pais in df['Country Name'].unique():
    df_pais = df[df['Country Name'] == pais]
    con_internet = df_pais[df_pais['tiene_internet'] == 'Sí']['interes_politica'].mean()
    sin_internet = df_pais[df_pais['tiene_internet'] == 'No']['interes_politica'].mean()
    if not (np.isnan(con_internet) or np.isnan(sin_internet)):
        diferencias.append({
            'País': pais,
            'Con Internet': con_internet,
            'Sin Internet': sin_internet,
            'Diferencia': con_internet - sin_internet
        })

diferencias_df = pd.DataFrame(diferencias).sort_values('Diferencia', ascending=False)
print("\n\nDiferencia en interés político (Con Internet - Sin Internet) por país:")
print(diferencias_df.to_string(index=False))


In [ ]:
# Visualización: Acceso a internet y participación política por país
fig, axes = plt.subplots(1, 2, figsize=(18, 8))

# 1. Porcentaje de acceso a internet por país
acceso_si = acceso_por_pais['Sí'].sort_values(ascending=True)
axes[0].barh(acceso_si.index, acceso_si.values, color='steelblue', alpha=0.8, edgecolor='black')
axes[0].set_xlabel('Porcentaje con Acceso a Internet (%)', fontweight='bold')
axes[0].set_title('Acceso a Internet por País en América Latina', fontweight='bold', fontsize=13)
axes[0].grid(axis='x', alpha=0.3)
for i, v in enumerate(acceso_si.values):
    axes[0].text(v, i, f' {v:.1f}%', va='center', fontweight='bold')

# 2. Diferencia en interés político (con vs sin internet)
diferencias_ordenadas = diferencias_df.sort_values('Diferencia', ascending=True)
colors = ['green' if x > 0 else 'red' for x in diferencias_ordenadas['Diferencia']]
axes[1].barh(diferencias_ordenadas['País'], diferencias_ordenadas['Diferencia'], 
             color=colors, alpha=0.7, edgecolor='black')
axes[1].axvline(0, color='black', linestyle='--', linewidth=1)
axes[1].set_xlabel('Diferencia en Interés Político\n(Con Internet - Sin Internet)', fontweight='bold')
axes[1].set_title('Brecha Digital en Interés Político por País', fontweight='bold', fontsize=13)
axes[1].grid(axis='x', alpha=0.3)
for i, v in enumerate(diferencias_ordenadas['Diferencia']):
    axes[1].text(v, i, f' {v:+.2f}', va='center', fontweight='bold')

plt.tight_layout()
plt.show()


### 4.3. Análisis de Correlación: Internet, Redes Sociales y Participación


In [ ]:
# Análisis de correlación entre variables digitales y participación
variables_digitales = ['acceso_internet', 'uso_redes', 'confianza_redes', 
                       'interes_politica', 'confianza_gobierno', 'satisfaccion_democracia']
df_digital = df[variables_digitales].copy()

# Convertir acceso_internet a numérico para correlación
df_digital['acceso_internet_num'] = df_digital['acceso_internet'].map({1: 1, 2: 0})

corr_digital = df_digital[['acceso_internet_num', 'uso_redes', 'confianza_redes', 
                           'interes_politica', 'confianza_gobierno', 'satisfaccion_democracia']].corr()

print("Matriz de correlación - Variables digitales y participación:")
print(corr_digital.round(3))

# Test de significancia
print("\n\nCorrelación entre acceso a internet e interés en política:")
df_clean_digital = df_digital[['acceso_internet_num', 'interes_politica']].dropna()
if len(df_clean_digital) > 0:
    corr_coef, p_value = stats.pearsonr(df_clean_digital['acceso_internet_num'], 
                                        df_clean_digital['interes_politica'])
    print(f"  Coeficiente de correlación: {corr_coef:.4f}")
    print(f"  Valor p: {p_value:.4f}")
    print(f"  Significativo (p < 0.05): {'Sí' if p_value < 0.05 else 'No'}")


In [ ]:
# Visualización: Heatmap de correlaciones
plt.figure(figsize=(10, 8))
mask = np.triu(np.ones_like(corr_digital, dtype=bool))
sns.heatmap(corr_digital, annot=True, fmt='.3f', cmap='coolwarm', center=0,
            square=True, linewidths=1, cbar_kws={'label': 'Coeficiente de Correlación'},
            mask=mask, vmin=-1, vmax=1)
plt.title('Correlaciones: Variables Digitales y Participación Política', 
          fontweight='bold', fontsize=14, pad=20)
plt.tight_layout()
plt.show()


### 4.4. Análisis Multivariado: Educación + Acceso Digital


In [ ]:
# Análisis combinado: Educación y acceso a internet
educ_internet = df.groupby(['grupo_educativo', 'tiene_internet']).agg({
    'interes_politica': 'mean',
    'confianza_gobierno': 'mean',
    'satisfaccion_democracia': 'mean'
}).round(2)

print("Interacción entre nivel educativo y acceso a internet:")
print(educ_internet)


In [ ]:
# Visualización: Interacción educación e internet
pivot_interes = df.pivot_table(
    values='interes_politica',
    index='grupo_educativo',
    columns='tiene_internet',
    aggfunc='mean'
)

pivot_confianza = df.pivot_table(
    values='confianza_gobierno',
    index='grupo_educativo',
    columns='tiene_internet',
    aggfunc='mean'
)

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# 1. Interés en política
sns.heatmap(pivot_interes, annot=True, fmt='.2f', cmap='YlOrRd', 
            cbar_kws={'label': 'Interés Promedio (1-4)'}, 
            linewidths=1, linecolor='black', ax=axes[0])
axes[0].set_title('Interés en Política: Educación × Acceso a Internet', 
                   fontweight='bold', fontsize=12)
axes[0].set_xlabel('Acceso a Internet', fontweight='bold')
axes[0].set_ylabel('Nivel Educativo', fontweight='bold')

# 2. Confianza en gobierno
sns.heatmap(pivot_confianza, annot=True, fmt='.2f', cmap='RdYlGn', 
            cbar_kws={'label': 'Confianza Promedio (1-10)'}, 
            linewidths=1, linecolor='black', ax=axes[1])
axes[1].set_title('Confianza en Gobierno: Educación × Acceso a Internet', 
                   fontweight='bold', fontsize=12)
axes[1].set_xlabel('Acceso a Internet', fontweight='bold')
axes[1].set_ylabel('Nivel Educativo', fontweight='bold')

plt.tight_layout()
plt.show()


## 5. Visualizaciones para Audiencia: Formuladores de Políticas Públicas

### 5.1. Identificación de la Audiencia

**Audiencia objetivo:** Formuladores de políticas públicas, funcionarios gubernamentales y organizaciones internacionales que trabajan en desarrollo democrático en América Latina.

**Necesidades de la audiencia:**
- Visualizaciones claras y accionables
- Comparaciones entre países
- Identificación de brechas y desigualdades
- Evidencia para justificar inversiones en políticas específicas
- Información que pueda comunicarse a tomadores de decisión

### 5.2. Dashboard de Resumen Ejecutivo


In [ ]:
# Crear dashboard ejecutivo con las principales conclusiones
fig = plt.figure(figsize=(20, 12))
gs = fig.add_gridspec(3, 3, hspace=0.3, wspace=0.3)

# 1. Confianza en gobierno por educación (arriba izquierda)
ax1 = fig.add_subplot(gs[0, 0])
educ_confianza = df.groupby('grupo_educativo')['confianza_gobierno'].mean().sort_values()
ax1.barh(educ_confianza.index, educ_confianza.values, color=['#e74c3c', '#f39c12', '#27ae60'], alpha=0.8)
ax1.set_xlabel('Confianza (1-10)', fontweight='bold', fontsize=10)
ax1.set_title('1. Educación y Confianza en Gobierno', fontweight='bold', fontsize=11)
ax1.grid(axis='x', alpha=0.3)
for i, v in enumerate(educ_confianza.values):
    ax1.text(v, i, f' {v:.2f}', va='center', fontsize=9)

# 2. Acceso a internet por país (arriba centro)
ax2 = fig.add_subplot(gs[0, 1])
acceso_si = acceso_por_pais['Sí'].sort_values(ascending=True).tail(10)
ax2.barh(acceso_si.index, acceso_si.values, color='steelblue', alpha=0.8)
ax2.set_xlabel('Acceso (%)', fontweight='bold', fontsize=10)
ax2.set_title('2. Acceso a Internet por País', fontweight='bold', fontsize=11)
ax2.grid(axis='x', alpha=0.3)
for i, v in enumerate(acceso_si.values):
    ax2.text(v, i, f' {v:.1f}%', va='center', fontsize=9)

# 3. Interés político: Internet vs Sin Internet (arriba derecha)
ax3 = fig.add_subplot(gs[0, 2])
internet_interes = df.groupby('tiene_internet')['interes_politica'].mean()
ax3.bar(internet_interes.index, internet_interes.values, color=['#e74c3c', '#27ae60'], alpha=0.8)
ax3.set_ylabel('Interés (1-4)', fontweight='bold', fontsize=10)
ax3.set_title('3. Brecha Digital en Interés Político', fontweight='bold', fontsize=11)
ax3.grid(axis='y', alpha=0.3)
for i, v in enumerate(internet_interes.values):
    ax3.text(i, v, f'\n{v:.2f}', ha='center', fontweight='bold', fontsize=10)

# 4. Heatmap confianza por país y educación (centro izquierda, más grande)
ax4 = fig.add_subplot(gs[1:, 0])
pivot_confianza = df.pivot_table(
    values='confianza_gobierno', 
    index='Country Name', 
    columns='grupo_educativo', 
    aggfunc='mean'
)
sns.heatmap(pivot_confianza, annot=True, fmt='.1f', cmap='RdYlGn', 
            cbar_kws={'label': 'Confianza (1-10)'}, linewidths=0.5, ax=ax4, fontsize=8)
ax4.set_title('4. Confianza en Gobierno: País × Educación', fontweight='bold', fontsize=11)
ax4.set_xlabel('Nivel Educativo', fontweight='bold', fontsize=10)
ax4.set_ylabel('País', fontweight='bold', fontsize=10)

# 5. Interacción educación e internet (centro derecha)
ax5 = fig.add_subplot(gs[1:, 1:])
pivot_interes = df.pivot_table(
    values='interes_politica',
    index='grupo_educativo',
    columns='tiene_internet',
    aggfunc='mean'
)
sns.heatmap(pivot_interes, annot=True, fmt='.2f', cmap='YlOrRd', 
            cbar_kws={'label': 'Interés (1-4)'}, linewidths=1, linecolor='black', ax=ax5)
ax5.set_title('5. Interés Político: Educación × Acceso a Internet', fontweight='bold', fontsize=11)
ax5.set_xlabel('Acceso a Internet', fontweight='bold', fontsize=10)
ax5.set_ylabel('Nivel Educativo', fontweight='bold', fontsize=10)

# Título general
fig.suptitle('Dashboard Ejecutivo: Educación, Brecha Digital y Participación Política en América Latina', 
             fontsize=16, fontweight='bold', y=0.98)

plt.show()


In [ ]:
# Comparación detallada de países seleccionados
paises_seleccionados = ['Uruguay', 'Chile', 'Argentina', 'Brasil', 'México', 'Colombia']
df_seleccion = df[df['Country Name'].isin(paises_seleccionados)]

fig, axes = plt.subplots(2, 2, figsize=(18, 12))

# 1. Confianza en gobierno por país y educación
pivot_pais_educ = df_seleccion.pivot_table(
    values='confianza_gobierno',
    index='Country Name',
    columns='grupo_educativo',
    aggfunc='mean'
)
pivot_pais_educ = pivot_pais_educ.reindex(paises_seleccionados)
pivot_pais_educ.plot(kind='bar', ax=axes[0, 0], color=['#e74c3c', '#f39c12', '#27ae60'], 
                     alpha=0.8, edgecolor='black', linewidth=1)
axes[0, 0].set_title('Confianza en Gobierno por País y Educación', fontweight='bold', fontsize=12)
axes[0, 0].set_ylabel('Confianza Promedio (1-10)', fontweight='bold')
axes[0, 0].set_xlabel('')
axes[0, 0].legend(title='Nivel Educativo', fontsize=9)
axes[0, 0].grid(axis='y', alpha=0.3)
axes[0, 0].tick_params(axis='x', rotation=45)

# 2. Acceso a internet por país
acceso_paises = df_seleccion.groupby('Country Name')['tiene_internet'].value_counts(normalize=True).unstack() * 100
acceso_paises = acceso_paises.reindex(paises_seleccionados)
acceso_paises['Sí'].plot(kind='bar', ax=axes[0, 1], color='steelblue', alpha=0.8, 
                         edgecolor='black', linewidth=1)
axes[0, 1].set_title('Porcentaje con Acceso a Internet', fontweight='bold', fontsize=12)
axes[0, 1].set_ylabel('Porcentaje (%)', fontweight='bold')
axes[0, 1].set_xlabel('')
axes[0, 1].grid(axis='y', alpha=0.3)
axes[0, 1].tick_params(axis='x', rotation=45)
for i, v in enumerate(acceso_paises['Sí']):
    axes[0, 1].text(i, v, f'\n{v:.1f}%', ha='center', fontweight='bold', fontsize=9)

# 3. Interés político: con vs sin internet por país
interes_internet = df_seleccion.groupby(['Country Name', 'tiene_internet'])['interes_politica'].mean().unstack()
interes_internet = interes_internet.reindex(paises_seleccionados)
x = np.arange(len(paises_seleccionados))
width = 0.35
axes[1, 0].bar(x - width/2, interes_internet['Sí'], width, label='Con Internet', 
                color='#27ae60', alpha=0.8, edgecolor='black')
axes[1, 0].bar(x + width/2, interes_internet['No'], width, label='Sin Internet', 
                color='#e74c3c', alpha=0.8, edgecolor='black')
axes[1, 0].set_title('Interés Político: Brecha Digital por País', fontweight='bold', fontsize=12)
axes[1, 0].set_ylabel('Interés Promedio (1-4)', fontweight='bold')
axes[1, 0].set_xlabel('')
axes[1, 0].set_xticks(x)
axes[1, 0].set_xticklabels(paises_seleccionados, rotation=45, ha='right')
axes[1, 0].legend()
axes[1, 0].grid(axis='y', alpha=0.3)

# 4. Satisfacción con democracia por país
satisfaccion_pais = df_seleccion.groupby('Country Name')['satisfaccion_democracia'].mean()
satisfaccion_pais = satisfaccion_pais.reindex(paises_seleccionados).sort_values()
axes[1, 1].barh(satisfaccion_pais.index, satisfaccion_pais.values, 
                color=plt.cm.RdYlGn(np.linspace(0.2, 0.8, len(satisfaccion_pais))), 
                alpha=0.8, edgecolor='black', linewidth=1)
axes[1, 1].set_title('Satisfacción con Democracia por País', fontweight='bold', fontsize=12)
axes[1, 1].set_xlabel('Satisfacción Promedio (1-10)', fontweight='bold')
axes[1, 1].grid(axis='x', alpha=0.3)
for i, v in enumerate(satisfaccion_pais.values):
    axes[1, 1].text(v, i, f' {v:.2f}', va='center', fontweight='bold', fontsize=9)

plt.suptitle('Análisis Comparativo: Países Seleccionados de América Latina', 
             fontsize=14, fontweight='bold', y=0.995)
plt.tight_layout()
plt.show()


## 6. Conclusiones y Recomendaciones para Políticas Públicas

### 6.1. Hallazgos Principales

**Pregunta 1: Educación y Participación Política**
- [Incluir aquí las conclusiones basadas en los análisis realizados]
- La correlación entre educación y confianza en instituciones sugiere...
- Las diferencias entre países indican...

**Pregunta 2: Brecha Digital y Participación Política**
- [Incluir aquí las conclusiones basadas en los análisis realizados]
- El acceso a internet está relacionado con...
- La brecha digital afecta diferencialmente a...

### 6.2. Recomendaciones de Política

1. **Políticas Educativas:**
   - [Recomendaciones basadas en hallazgos]

2. **Políticas de Inclusión Digital:**
   - [Recomendaciones basadas en hallazgos]

3. **Políticas de Fortalecimiento Democrático:**
   - [Recomendaciones basadas en hallazgos]


In [ ]:
# Resumen estadístico final
print("="*70)
print("RESUMEN EJECUTIVO DEL ANÁLISIS")
print("="*70)

print("\n1. EDUCACIÓN Y PARTICIPACIÓN POLÍTICA")
print("-" * 70)
df_clean_educ = df[['anos_estudio', 'confianza_gobierno']].dropna()
if len(df_clean_educ) > 0:
    corr_educ = stats.pearsonr(df_clean_educ['anos_estudio'], df_clean_educ['confianza_gobierno'])[0]
    print(f"Correlación educación-confianza gobierno: {corr_educ:.3f}")
    
    conf_primaria = df[df['grupo_educativo']=='Primaria (0-6)']['confianza_gobierno'].mean()
    conf_universidad = df[df['grupo_educativo']=='Universidad (13+)']['confianza_gobierno'].mean()
    if not (np.isnan(conf_primaria) or np.isnan(conf_universidad)):
        print(f"Diferencia confianza (Universidad vs Primaria): {conf_universidad - conf_primaria:.2f} puntos")

print("\n2. BRECHA DIGITAL Y PARTICIPACIÓN POLÍTICA")
print("-" * 70)
total_internet = df['tiene_internet'].notna().sum()
con_internet = (df['tiene_internet']=='Sí').sum()
if total_internet > 0:
    print(f"Porcentaje con acceso a internet: {con_internet / total_internet * 100:.1f}%")
    
    interes_con = df[df['tiene_internet']=='Sí']['interes_politica'].mean()
    interes_sin = df[df['tiene_internet']=='No']['interes_politica'].mean()
    if not (np.isnan(interes_con) or np.isnan(interes_sin)):
        print(f"Diferencia interés político (Con vs Sin Internet): {interes_con - interes_sin:.2f} puntos")
        
    df_clean_digital = df_digital[['acceso_internet_num', 'interes_politica']].dropna()
    if len(df_clean_digital) > 0:
        corr_digital = stats.pearsonr(df_clean_digital['acceso_internet_num'], 
                                      df_clean_digital['interes_politica'])[0]
        print(f"Correlación acceso internet-interés político: {corr_digital:.3f}")

print("\n3. INTERACCIÓN EDUCACIÓN × ACCESO DIGITAL")
print("-" * 70)
print("Interés político promedio:")
for educ in ['Primaria (0-6)', 'Secundaria (7-12)', 'Universidad (13+)']:
    for internet in ['Sí', 'No']:
        valor = df[(df['grupo_educativo']==educ) & (df['tiene_internet']==internet)]['interes_politica'].mean()
        if not np.isnan(valor):
            print(f"  {educ} + {internet} Internet: {valor:.2f}")

print("\n" + "="*70)
